## OPTUNA: 

Optuna es una biblioteca de optimización automática de hiperparámetros. El objetivo principal es encontrar la mejor combinación de valores de los hiperparámetros, para aumentar la precisión del modelo. 

Se definen los hiperparámetros a optimizar, como el tamaño del lote, la tasa de aprendizaje, los filtros de las capas convolutivas, las neuronas de las capas densas y la funcion de activacion, que en este caso son:
- ReLU: Devuelve el valor de entrada si es positivo o cero si es negativo
- Leaky ReLU: Variante de la ReLU, que en vez de convertir los números negativos a cero, los hace muy pequeños, para que no se "apaguen" por completo las neuronas, para que no se desactiven por completo, lo cual hace que deje de contribuir en el aprendizaje.

Para el estudio con optuna, se hace una optimización con 20 trials, en las que se van probando distintas combinaciones de hiperparámetros, y al final se imprimen los mejores hiperparámetros encontrados. Se crea una especie de estudio, donde se registrarámn los trials y sus resultados, y al poner direction="maximize", le estamos indicando que el objetivo es aumentar la precisión.  

A parte de esto, tenemos la función objetivo, que es la función principal de Optuna, que entrena el modelo de la red con las distintas combinaciones de hiperparámetros, y devuelve la precisión del modelo en el conjunto de prueba. El entrenamiento funciona de la siguiente manera: para cada conjunto de hiperparámetros, el modelo se entrena y calcula la precisión en el conjunto de prueba final. Optuna realiza 20 trials, que son entrenamientos con distintas combinaciones de hiperparámetros, y va ajustando los hiperparámetros. 

Los resultados se devuelven al final de la función objective, imprimiendo la precisión del conjunto de prueba, y optuna usa este valor para ver cóm de buenos son los hiperparámetros que se están probando.

Entre las ventajas de Optuna está la optimización automática de los hiperparámetros, ya que manualemnte es poco eficiente y mucho más trabajo. Además de ello usa estrategias más avanzadas como la búsqueda aleatoria y el pruning, que es una técnica que detiene los ensayos que no muestranb progreso prometedor temprano, lo cual ahorra tiempo y recursos ya que evita entrenamientos innecesarios, y permite encontrar soluciones óptimas más rápido.



In [ ]:
import optuna
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F

# Definir la función objetivo para Optuna
def objective(trial):
    # Hiperparámetros para optimizar
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    lr = trial.suggest_loguniform('lr', 0.001, 0.1)
    patience = trial.suggest_int('patience', 3, 10)
    num_filters_1 = trial.suggest_int('num_filters_1', 16, 64, step=16)
    num_filters_2 = trial.suggest_int('num_filters_2', 32, 128, step=32)
    num_filters_3 = trial.suggest_int('num_filters_3', 64, 256, step=64)
    num_neurons_1 = trial.suggest_int('num_neurons_1', 64, 256, step=64)
    num_neurons_2 = trial.suggest_int('num_neurons_2', 32, 128, step=32)
    activation_function = trial.suggest_categorical('activation_function', ['relu', 'leaky_relu'])

    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar
        transforms.RandomHorizontalFlip(p=0.5),  # Voltear horizontalmente
        transforms.RandomRotation(20),  # Rotar aleatoriamente hasta 20 grados
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Recortar aleatoriamente
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Cambios en color
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Redimensionar
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
    ])

    train_dataset = datasets.ImageFolder(root='dataset_animales/animals/animals', transform=train_transform)
    test_dataset = datasets.ImageFolder(root='dataset_animales/animals/animals', transform=test_transform)

    train_size = int(0.8 * len(train_dataset))
    test_size = len(train_dataset) - train_size

    train_data, _ = random_split(train_dataset, [train_size, test_size])
    _, test_data = random_split(test_dataset, [train_size, test_size])

    # Guardar los datos para entrenar y testear
    train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_data = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Modelo con tres capas convolutivas
    class ConvNet(nn.Module):
        def __init__(self):
            super(ConvNet, self).__init__()
            self.conv1 = nn.Conv2d(3, num_filters_1, kernel_size=3, stride=1, padding=1)
            self.conv2 = nn.Conv2d(num_filters_1, num_filters_2, kernel_size=3, stride=1, padding=1)
            self.conv3 = nn.Conv2d(num_filters_2, num_filters_3, kernel_size=3, stride=1, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
            self.dropout = nn.Dropout(0.5)
            self.fc1 = nn.Linear(num_filters_3 * 28 * 28, num_neurons_1)
            self.fc2 = nn.Linear(num_neurons_1, num_neurons_2)
            self.fc3 = nn.Linear(num_neurons_2, 7)
            self.activation_function = activation_function

        def forward(self, x):
            activation = F.relu if self.activation_function == 'relu' else F.leaky_relu
            x = self.pool(activation(self.conv1(x)))
            x = self.pool(activation(self.conv2(x)))
            x = self.pool(activation(self.conv3(x)))
            x = x.view(x.size(0), -1)
            x = self.dropout(activation(self.fc1(x)))
            x = activation(self.fc2(x))
            x = self.fc3(x) 
            return x

    model = ConvNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Entrenamiento
    best_test_accuracy = 0

    # Listas para guardar loss y accuracy
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    epochs = 10
    for epoch in range(epochs):
        # Train
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for images, labels in train_data:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_data)
        train_accuracy = 100 * correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Test
        model.eval() 
        running_loss = 0.0
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_data:
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_loss = running_loss / len(test_data)
        test_accuracy = 100 * correct_test / total_test
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        # Mostrar el resultado al final de cada época
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'Epoch {epoch+1}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        # Actualizar mejor precisión de validación
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy

    return best_test_accuracy

# Crear estudio Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Imprimir los mejores hiperparámetros
print('Best hyperparameters:', study.best_params)
